In [1]:
import nltk
from nltk.corpus import brown
from nltk import bigrams, ConditionalFreqDist

In [89]:
import nltk
from nltk.corpus import brown
from nltk import bigrams, ConditionalFreqDist

sentences = brown.sents()

tagged_sentences = [nltk.pos_tag(sentence) for sentence in sentences]

# Convert the sentences to sequences of POS tags
# tag_sequences = [[tag for word, tag in sentence] for sentence in tagged_sentences]

# # Generate bigrams from these sequences
# tag_bigrams = [bigram for sent in tag_sequences for bigram in bigrams(sent)]

# # Train a conditional frequency distribution model on these bigrams
# cfd = ConditionalFreqDist(tag_bigrams)

# # Define a function to calculate the probability of a tag given the previous tag
# def tag_probability(prev_tag, current_tag):
#     # Count of current tag following prev_tag
#     current_tag_count = cfd[prev_tag][current_tag]
#     # Total count of all tags following prev_tag
#     total_count = sum(cfd[prev_tag].values())
#     probability = current_tag_count / total_count if total_count > 0 else 0
#     return probability

# # Example usage: Probability of a noun (NOUN) following a determiner (DET)
# print(tag_probability('DET', 'NOUN'))
# print()


In [104]:
tag_sequences = [[tag for word, tag in sentence] for sentence in tagged_sentences]
tag_bigrams = [bigram for sent in tag_sequences for bigram in bigrams(sent)]
cfd = ConditionalFreqDist(tag_bigrams)

# Define a function to calculate the probability of a tag given the previous tag
def tag_probability(prev_tag, current_tag):
    # Count of current tag following prev_tag
    current_tag_count = cfd[prev_tag][current_tag]
    # Total count of all tags following prev_tag
    total_count = sum(cfd[prev_tag].values())
    probability = current_tag_count / total_count if total_count > 0 else 0
    return probability

# Example usage: Probability of a noun (NOUN) following a determiner (DET)
print(tag_probability('TO', 'VBP'))

7.646136789387162e-05


In [105]:
import pickle

# Save the cfd object to a file
with open('cfd.pkl', 'wb') as f:
    pickle.dump(cfd, f)

In [2]:
import pickle

# Load the cfd object from the file
with open('cfd.pkl', 'rb') as f:
    cfd = pickle.load(f)

In [3]:
# Define a function to calculate the probability of a tag given the previous tag
def tag_probability(prev_tag, current_tag):
    # Count of current tag following prev_tag
    current_tag_count = cfd[prev_tag][current_tag]
    # Total count of all tags following prev_tag
    total_count = sum(cfd[prev_tag].values())
    probability = current_tag_count / total_count if total_count > 0 else 0
    return probability

# Example usage: Probability of a noun (NOUN) following a determiner (DET)
print(tag_probability('TO', 'VBP'))

7.646136789387162e-05


In [102]:
nltk.pos_tag(['to', 'be',])

[('to', 'TO'), ('be', 'VB')]

In [70]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def get_pos_tags(text):
    """
    Tokenize the input text and return POS tags for each token.
    
    Parameters:
    - text (str): The input text to tag.
    
    Returns:
    - list of tuples: A list where each tuple contains a token and its POS tag.
    """
    # Tokenize the text
    tokens = word_tokenize(text)
    # Tag the tokens with POS tags
    tagged_tokens = pos_tag(tokens, tagset='brown')
    return tagged_tokens

# Example text
text = "i am sun"

# Get POS tags for the example text
tags = get_pos_tags(text)
print(tags)


[('i', 'UNK'), ('am', 'UNK'), ('sun', 'UNK')]


In [71]:
import spacy
from collections import Counter

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Load the POS-tagged sentences from the Brown Corpus using spaCy
brown_corpus_sentences = brown.sents(categories='news')[:100]  # Using a subset of sentences for demonstration
tagged_sentences = []
for sent in brown_corpus_sentences:
    doc = nlp(" ".join(sent))
    tagged_sentences.append([(token.text, token.pos_) for token in doc])

# Convert the sentences to sequences of POS tags
tag_sequences = [[tag for word, tag in sentence] for sentence in tagged_sentences]

# Generate bigrams from these sequences
tag_bigrams = [bigram for sent in tag_sequences for bigram in zip(sent[:-1], sent[1:])]

# Count occurrences of bigrams
cfd = Counter(tag_bigrams)

# Define a function to calculate the probability of a tag given the previous tag
def tag_probability(prev_tag, current_tag):
    # Count of current tag following prev_tag
    current_tag_count = cfd[(prev_tag, current_tag)]
    # Total count of all tags following prev_tag
    total_count = sum(cfd[bigram] for bigram in cfd if bigram[0] == prev_tag)
    probability = current_tag_count / total_count if total_count > 0 else 0
    return probability

# Example usage: Probability of a noun (NOUN) following a determiner (DET)
print(tag_probability('DET', 'NOUN'))


0.6337448559670782


In [81]:
import spacy
from collections import defaultdict
import nltk
from nltk.corpus import brown

# Download the Brown Corpus if it's not already downloaded
# nltk.download('brown')

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the Brown Corpus as a list of sentences
sentences = brown.sents()

# Initialize a dictionary to hold our sequence data
# Using defaultdict to automatically initialize unseen keys with an empty list
pos_sequences = defaultdict(list)

# Process each sentence with spaCy
for sentence in sentences:
    # Join the words in the sentence into a single string
    text = ' '.join(sentence)
    # Process the text with spaCy
    doc = nlp(text)
    # Iterate through the tokens in the doc
    for token, next_token in zip(doc[:-1], doc[1:]):
        # Append the POS of the next token to the list of the current token's POS
        pos_sequences[token.pos_].append(next_token.pos_)
        
# If you want each POS tag to map to a unique list of following POS tags:
for pos in pos_sequences:
    pos_sequences[pos] = list(set(pos_sequences[pos]))

print(pos_sequences)

defaultdict(<class 'list'>, {'DET': ['ADJ', 'X', 'SYM', 'AUX', 'INTJ', 'NOUN', 'VERB', 'NUM', 'PRON', 'ADP', 'PROPN', 'PART', 'ADV', 'SCONJ', 'PUNCT', 'DET', 'CCONJ'], 'PROPN': ['ADJ', 'X', 'SYM', 'AUX', 'INTJ', 'NOUN', 'VERB', 'NUM', 'PRON', 'ADP', 'PROPN', 'PART', 'ADV', 'SCONJ', 'PUNCT', 'DET', 'CCONJ'], 'VERB': ['ADJ', 'X', 'SYM', 'AUX', 'INTJ', 'NOUN', 'VERB', 'NUM', 'PRON', 'ADP', 'PROPN', 'PART', 'ADV', 'SCONJ', 'PUNCT', 'DET', 'CCONJ'], 'NOUN': ['ADJ', 'X', 'SYM', 'AUX', 'INTJ', 'NOUN', 'VERB', 'NUM', 'PRON', 'ADP', 'PROPN', 'PART', 'ADV', 'SCONJ', 'PUNCT', 'DET', 'CCONJ'], 'ADP': ['ADJ', 'X', 'SYM', 'AUX', 'INTJ', 'NOUN', 'VERB', 'NUM', 'PRON', 'ADP', 'PROPN', 'PART', 'ADV', 'SCONJ', 'PUNCT', 'DET', 'CCONJ'], 'PART': ['ADJ', 'SYM', 'AUX', 'INTJ', 'NOUN', 'VERB', 'NUM', 'PRON', 'ADP', 'PROPN', 'PART', 'ADV', 'SCONJ', 'PUNCT', 'DET', 'CCONJ'], 'ADJ': ['ADJ', 'X', 'SYM', 'AUX', 'INTJ', 'NOUN', 'VERB', 'NUM', 'PRON', 'ADP', 'PROPN', 'PART', 'ADV', 'SCONJ', 'PUNCT', 'DET', 'CCONJ']

In [82]:
import json

# ... (rest of your code)

# Save the pos_sequences dictionary to a JSON file
with open('pos_sequences.json', 'w') as f:
    json.dump(pos_sequences, f)

In [85]:
text = "i want be to a good person"
doc = nlp(text)
for token in doc:
    print(token.text, token.pos_)

i PRON
want VERB
be AUX
to ADP
a DET
good ADJ
person NOUN
